# Multiplicación de un archivo

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

conf = (SparkConf().set("spark.executor.memory", "6g")   #<--- Por executor. Como hay dos instancias, la mitad por executor.
                    .set("spark.executor.cores", "3")    #<--- Por cada executor. Como hay 4, 4*3=12 cores se usan.
                    .set("spark.executor.instances", "2")
                    .set("spark.eventLog.enabled", "true"))
spark = SparkSession.builder.master("spark://spark-master:7077").appName("Multiplica").config(conf=conf).getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/08 18:44:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
%%time
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
import pyspark.sql.functions as F
from pyspark.sql.functions import *

df = spark.read.csv("./Fotolescu/useful-master/kaggle/coronavirus-forecasting/data/train.csv", sep=",", header=True, inferSchema=True)

# shape:
print(80*'=','\nfilas --> ', df.count(), 'columnas --> ', len(df.columns), '\n', 80*'=')
df.show(truncate=False)

#df = df.withColumn("proc", substring("ctId",1,6))
#df.write.mode("append").partitionBy("proc").parquet("./datos/fakeContracts01.parquet")

filas -->  16756 columnas -->  8 
+---+--------------+--------------+----+----+----------+--------------+----------+
|Id |Province/State|Country/Region|Lat |Long|Date      |ConfirmedCases|Fatalities|
+---+--------------+--------------+----+----+----------+--------------+----------+
|1  |null          |Afghanistan   |33.0|65.0|2020-01-22|0.0           |0.0       |
|2  |null          |Afghanistan   |33.0|65.0|2020-01-23|0.0           |0.0       |
|3  |null          |Afghanistan   |33.0|65.0|2020-01-24|0.0           |0.0       |
|4  |null          |Afghanistan   |33.0|65.0|2020-01-25|0.0           |0.0       |
|5  |null          |Afghanistan   |33.0|65.0|2020-01-26|0.0           |0.0       |
|6  |null          |Afghanistan   |33.0|65.0|2020-01-27|0.0           |0.0       |
|7  |null          |Afghanistan   |33.0|65.0|2020-01-28|0.0           |0.0       |
|8  |null          |Afghanistan   |33.0|65.0|2020-01-29|0.0           |0.0       |
|9  |null          |Afghanistan   |33.0|65.0|2020-01-

In [3]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- ConfirmedCases: double (nullable = true)
 |-- Fatalities: double (nullable = true)



### Crea una columa con un string explodable 10 veces

In [9]:
from pyspark.sql.functions import lit
#df = df.withColumn("forEx", lit("1|2|3|4|5|6|7|8|9|0"))
df = df.withColumn("forEx", lit("1 2 3 4 5 6 7 8 9 0"))

df.show(truncate=False)

+---+--------------+--------------+----+----+----------+--------------+----------+-------------------+
|Id |Province/State|Country/Region|Lat |Long|Date      |ConfirmedCases|Fatalities|forEx              |
+---+--------------+--------------+----+----+----------+--------------+----------+-------------------+
|1  |null          |Afghanistan   |33.0|65.0|2020-01-22|0.0           |0.0       |1 2 3 4 5 6 7 8 9 0|
|2  |null          |Afghanistan   |33.0|65.0|2020-01-23|0.0           |0.0       |1 2 3 4 5 6 7 8 9 0|
|3  |null          |Afghanistan   |33.0|65.0|2020-01-24|0.0           |0.0       |1 2 3 4 5 6 7 8 9 0|
|4  |null          |Afghanistan   |33.0|65.0|2020-01-25|0.0           |0.0       |1 2 3 4 5 6 7 8 9 0|
|5  |null          |Afghanistan   |33.0|65.0|2020-01-26|0.0           |0.0       |1 2 3 4 5 6 7 8 9 0|
|6  |null          |Afghanistan   |33.0|65.0|2020-01-27|0.0           |0.0       |1 2 3 4 5 6 7 8 9 0|
|7  |null          |Afghanistan   |33.0|65.0|2020-01-28|0.0           |0.

### Genera 10 filas por fila basado en el nuevo campo

In [27]:
from pyspark.sql.functions import split, explode
df2 = df.withColumn('forEx',explode(split('forEx',' ')))
df2 = df2.withColumn('Id', ((col('forEx') * 100000)+col('Id')).cast('int'))
print(df2.count())

df2.show(truncate=False)

167560
+------+--------------+--------------+----+----+----------+--------------+----------+-----+
|Id    |Province/State|Country/Region|Lat |Long|Date      |ConfirmedCases|Fatalities|forEx|
+------+--------------+--------------+----+----+----------+--------------+----------+-----+
|100001|null          |Afghanistan   |33.0|65.0|2020-01-22|0.0           |0.0       |1    |
|200001|null          |Afghanistan   |33.0|65.0|2020-01-22|0.0           |0.0       |2    |
|300001|null          |Afghanistan   |33.0|65.0|2020-01-22|0.0           |0.0       |3    |
|400001|null          |Afghanistan   |33.0|65.0|2020-01-22|0.0           |0.0       |4    |
|500001|null          |Afghanistan   |33.0|65.0|2020-01-22|0.0           |0.0       |5    |
|600001|null          |Afghanistan   |33.0|65.0|2020-01-22|0.0           |0.0       |6    |
|700001|null          |Afghanistan   |33.0|65.0|2020-01-22|0.0           |0.0       |7    |
|800001|null          |Afghanistan   |33.0|65.0|2020-01-22|0.0           

### Damos un vistazo rápido al contenido del archivo

In [23]:
df2.describe().show(truncate=False)

+-------+------------------+--------------+--------------+------------------+-----------------+----------+------------------+------------------+-----------------+
|summary|Id                |Province/State|Country/Region|Lat               |Long             |Date      |ConfirmedCases    |Fatalities        |forEx            |
+-------+------------------+--------------+--------------+------------------+-----------------+----------+------------------+------------------+-----------------+
|count  |167560            |76700         |167560        |166970            |166970           |167560    |167560            |167560            |167560           |
|mean   |463189.5          |null          |null          |26.337324028271926|5.039602826852433|null      |271.7501193602292 |9.221532585342564 |4.5              |
|stddev |287330.16738948796|null          |null          |22.962444102628087|79.93311275750844|null      |3261.0647213994102|141.52078096346398|2.872289894211314|
|min    |1            

In [29]:
%%time
for c in df2.columns:df2.groupBy(c).count().show(100, truncate=False)

+------+-----+
|Id    |count|
+------+-----+
|400003|1    |
|900004|1    |
|500025|1    |
|700037|1    |
|700097|1    |
|400129|1    |
|700139|1    |
|148   |1    |
|400152|1    |
|300282|1    |
|400301|1    |
|200379|1    |
|700410|1    |
|800467|1    |
|471   |1    |
|300474|1    |
|496   |1    |
|300522|1    |
|300574|1    |
|500597|1    |
|300601|1    |
|400612|1    |
|800614|1    |
|600660|1    |
|200687|1    |
|300779|1    |
|400797|1    |
|100800|1    |
|700861|1    |
|800870|1    |
|200878|1    |
|100884|1    |
|500942|1    |
|400982|1    |
|100986|1    |
|201031|1    |
|101055|1    |
|201148|1    |
|401161|1    |
|1238  |1    |
|501247|1    |
|501310|1    |
|901322|1    |
|1342  |1    |
|901355|1    |
|601404|1    |
|701412|1    |
|801421|1    |
|501433|1    |
|601529|1    |
|801532|1    |
|301535|1    |
|901541|1    |
|1591  |1    |
|801611|1    |
|101627|1    |
|601681|1    |
|201694|1    |
|801701|1    |
|101775|1    |
|601775|1    |
|401778|1    |
|801785|1    |
|301798|1 

### Cuenta cuántos nan or null hay en cada columna:

In [6]:
print(df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show())

+---+--------------+--------------+---+----+----+--------------+----------+
| Id|Province/State|Country/Region|Lat|Long|Date|ConfirmedCases|Fatalities|
+---+--------------+--------------+---+----+----+--------------+----------+
|  0|          9086|             0| 59|  59|   0|             0|         0|
+---+--------------+--------------+---+----+----+--------------+----------+

None


In [7]:
df.filter(col('Province/State').isNull()).show()

+---+--------------+--------------+----+----+----------+--------------+----------+
| Id|Province/State|Country/Region| Lat|Long|      Date|ConfirmedCases|Fatalities|
+---+--------------+--------------+----+----+----------+--------------+----------+
|  1|          null|   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|
|  2|          null|   Afghanistan|33.0|65.0|2020-01-23|           0.0|       0.0|
|  3|          null|   Afghanistan|33.0|65.0|2020-01-24|           0.0|       0.0|
|  4|          null|   Afghanistan|33.0|65.0|2020-01-25|           0.0|       0.0|
|  5|          null|   Afghanistan|33.0|65.0|2020-01-26|           0.0|       0.0|
|  6|          null|   Afghanistan|33.0|65.0|2020-01-27|           0.0|       0.0|
|  7|          null|   Afghanistan|33.0|65.0|2020-01-28|           0.0|       0.0|
|  8|          null|   Afghanistan|33.0|65.0|2020-01-29|           0.0|       0.0|
|  9|          null|   Afghanistan|33.0|65.0|2020-01-30|           0.0|       0.0|
| 10

In [8]:
%%time
spark.sparkContext.stop()


CPU times: user 1.4 ms, sys: 223 µs, total: 1.62 ms
Wall time: 107 ms
